In [ ]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.1 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏

In [ ]:
%pip install faiss-cpu

In [ ]:
!pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install -q gradio torch soundfile transformers

In [ ]:
# # Install required packages (run once)
# !pip install -q gradio torch soundfile transformers pydub

In [ ]:
# ===== Imports =====
import gradio as gr
import torch
import soundfile as sf
import tempfile
from pydub import AudioSegment
from STT import SaudiSTT
from ArabicRAGGPT import ArabicRAGGPT_class
from transformers import VitsModel, AutoTokenizer
import os

# ===== Load TTS =====
@torch.no_grad()
def load_tts_model():
    model_id = "facebook/mms-tts-ara"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = VitsModel.from_pretrained(model_id)
    return tokenizer, model

tts_tokenizer, tts_model = load_tts_model()

# ===== Convert audio to WAV 16kHz mono =====
def convert_to_wav(filepath):
    audio = AudioSegment.from_file(filepath)
    audio = audio.set_channels(1).set_frame_rate(16000)
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    audio.export(tmp_file.name, format="wav")
    return tmp_file.name

# ===== Text to Speech =====
def text_to_speech(text):
    inputs = tts_tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        wav = tts_model(**inputs).waveform
    wav = wav.squeeze().cpu().numpy().astype("float32")
    rate = tts_model.config.sampling_rate if hasattr(tts_model.config, "sampling_rate") else 22050
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(tmp_file.name, wav, rate)
    return tmp_file.name

# ===== Robust STT Fallback =====
class RobustSTT:
    def __init__(self):
        self.stt_model = None
        self.load_stt()

    def load_stt(self):
        try:
            print("🔄 جاري تحميل SaudiSTT...")
            self.stt_model = SaudiSTT()

            if hasattr(self.stt_model, 'transcribe_audio'):
                print("✅ SaudiSTT loaded successfully")
                return True
            else:
                print("❌ SaudiSTT missing transcribe_audio method")
                self.stt_model = None
                return False

        except Exception as e:
            print(f"❌ Failed to load SaudiSTT: {e}")
            self.stt_model = None
            return False

    def transcribe_audio(self, audio_path):
        try:
            if self.stt_model and hasattr(self.stt_model, 'transcribe_audio'):
                result = self.stt_model.transcribe_audio(audio_path)
                if result and result.strip():
                    return result

            return "هذا نص تجريبي من نظام STT الاحتياطي. يبدو أن النظام الرئيسي غير متوفر حالياً."

        except Exception as e:
            print(f"❌ STT transcription error: {e}")
            return f"نظام التعرف على الصوت يواجه مشكلة تقنية: {str(e)}"

# ===== Initialize Models =====
def initialize_models():
    models = {}

    # Initialize STT with robust fallback
    print("🔄 تهيئة نظام التعرف على الصوت...")
    models['stt'] = RobustSTT()

    # RAG - استخدام الفايل الموجود فقط
    try:
        print("🔄 جاري تحميل RAG...")
        api_key = "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"

        possible_paths = "/content/info.md"
        info_path = None

        if os.path.exists(possible_paths):
            print("i find the file")
            info_path = possible_paths

        # البحث عن ملف info.md
        # for path in possible_paths:
        # if os.path.exists(path):
        #         info_path = path


        if info_path:
            print(f"✅ Found info.md at: {info_path}")
            rag_system = ArabicRAGGPT_class(info_path, api_key)
            # تهيئة نظام RAG فوراً
            rag_system.initialize()
            models['rag'] = rag_system
            print("✅ RAG loaded and initialized successfully")
        else:
            print("❌ info.md file not found")
            # بدون إنشاء sample - نستخدم fallback بسيط
            class SimpleRAG:
                def ask_question(self, question):
                    return "❌ ملف info.md غير موجود. يرجى التأكد من وجود الملف في المسار الصحيح."
            models['rag'] = SimpleRAG()

    except Exception as e:
        print(f"❌ RAG loading failed: {e}")
        class SimpleRAG:
            def ask_question(self, question):
                return f"خطأ في تحميل نظام RAG: {str(e)}"
        models['rag'] = SimpleRAG()

    return models

# ===== Pipeline =====
def pipeline(audio_file):
    # Step 0: Convert audio file
    try:
        if isinstance(audio_file, tuple):
            audio_file = audio_file[0]
        wav_path = convert_to_wav(audio_file)
    except Exception as e:
        error_msg = f"Audio Conversion Error: {e}"
        return error_msg, "", None

    # Step 1: STT
    try:
        text = models['stt'].transcribe_audio(wav_path)
        if not text or text.strip() == "":
            text = "❌ لم يتم التعرف على أي نص في التسجيل الصوتي"
    except Exception as e:
        text = f"STT Error: {e}"

    # Step 2: RAG
    try:
        answer = models['rag'].ask_question(text)
    except Exception as e:
        answer = f"RAG Error: {e}"

    # Step 3: TTS
    try:
        audio_out = text_to_speech(answer)
    except Exception as e:
        audio_out = None
        print(f"TTS Error: {e}")

    # Clean up temporary files
    try:
        os.unlink(wav_path)
    except:
        pass

    return text, answer, audio_out

# ===== Init Models =====
print("🚀 Starting model initialization...")
models = initialize_models()

# Print final status
print("\n" + "="*50)
print("📊 Final Model Status:")
print(f"STT: {'✅ Loaded' if models['stt'].stt_model is not None else '⚠️ Using Fallback'}")
print(f"RAG: {'✅ Loaded' if hasattr(models['rag'], 'ask_question') else '❌ Failed'}")
print("="*50 + "\n")

# ===== Gradio Interface =====
iface = gr.Interface(
    fn=pipeline,
    inputs=gr.Audio(type="filepath", label="ارفع ملف صوت"),
    outputs=[
        gr.Textbox(label="📄 النص المستخرج (STT)"),
        gr.Textbox(label="🤖 إجابة الموديل (RAG)"),
        gr.Audio(label="🔊 الصوت الناتج (TTS)")
    ],
    title="🎙️ STT → RAG → TTS Demo",
    description="ارفع ملف صوت ليقوم النظام بتحويله لنص، ثم الإجابة عليه، ثم توليد الصوت"
)

print("🎉 Launching Gradio Interface...")
iface.launch(share=True)

🚀 Starting model initialization...
🔄 تهيئة نظام التعرف على الصوت...
🔄 جاري تحميل SaudiSTT...
✅ SaudiSTT loaded successfully
🔄 جاري تحميل RAG...
i find the file
✅ Found info.md at: /content/info.md
Time : 0.00
✅ RAG loaded and initialized successfully

📊 Final Model Status:
STT: ✅ Loaded
RAG: ✅ Loaded

🎉 Launching Gradio Interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a2c82ad4c9af4eadaf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Try A new Thing

In [ ]:
pip install  transformers[torch]

SeyedAli/Arabic-Speech-synthesis



In [ ]:
# ===== Imports =====
import gradio as gr
import torch
import soundfile as sf
import tempfile
from pydub import AudioSegment
from STT import SaudiSTT
from ArabicRAGGPT import ArabicRAGGPT_class
from transformers import VitsModel, AutoTokenizer
import os

# ===== Load TTS =====
@torch.no_grad()
def load_tts_model():
    # model_id = "facebook/mms-tts-ara"
    # model_id = "wasmdashai/vits-ar"
    # model_id ="facebook/mms-tts-uig-script_arabic"
    model_id = "SeyedAli/Arabic-Speech-synthesis"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = VitsModel.from_pretrained(model_id)
    return tokenizer, model

tts_tokenizer, tts_model = load_tts_model()

# ===== Convert audio to WAV 16kHz mono =====
def convert_to_wav(filepath):
    audio = AudioSegment.from_file(filepath)
    audio = audio.set_channels(1).set_frame_rate(16000)
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    audio.export(tmp_file.name, format="wav")
    return tmp_file.name

# ===== Text to Speech =====
def text_to_speech(text):
    inputs = tts_tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        wav = tts_model(**inputs).waveform
    wav = wav.squeeze().cpu().numpy().astype("float32")
    rate = tts_model.config.sampling_rate if hasattr(tts_model.config, "sampling_rate") else 22050
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(tmp_file.name, wav, rate)
    return tmp_file.name

# ===== Robust STT Fallback =====
class RobustSTT:
    def __init__(self):
        self.stt_model = None
        self.load_stt()

    def load_stt(self):
        try:
            print("🔄 جاري تحميل SaudiSTT...")
            self.stt_model = SaudiSTT()

            if hasattr(self.stt_model, 'transcribe_audio'):
                print("✅ SaudiSTT loaded successfully")
                return True
            else:
                print("❌ SaudiSTT missing transcribe_audio method")
                self.stt_model = None
                return False

        except Exception as e:
            print(f"❌ Failed to load SaudiSTT: {e}")
            self.stt_model = None
            return False

    def transcribe_audio(self, audio_path):
        try:
            if self.stt_model and hasattr(self.stt_model, 'transcribe_audio'):
                result = self.stt_model.transcribe_audio(audio_path)
                if result and result.strip():
                    return result

            return "هذا نص تجريبي من نظام STT الاحتياطي. يبدو أن النظام الرئيسي غير متوفر حالياً."

        except Exception as e:
            print(f"❌ STT transcription error: {e}")
            return f"نظام التعرف على الصوت يواجه مشكلة تقنية: {str(e)}"

# ===== Initialize Models =====
def initialize_models():
    models = {}

    # Initialize STT with robust fallback
    print("🔄 تهيئة نظام التعرف على الصوت...")
    models['stt'] = RobustSTT()

    # RAG - استخدام الفايل الموجود فقط
    try:
        print("🔄 جاري تحميل RAG...")
        api_key = "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"

        possible_paths = "/content/info.md"
        info_path = None

        if os.path.exists(possible_paths):
            print("i find the file")
            info_path = possible_paths

        # البحث عن ملف info.md
        # for path in possible_paths:
        # if os.path.exists(path):
        #         info_path = path


        if info_path:
            print(f"✅ Found info.md at: {info_path}")
            rag_system = ArabicRAGGPT_class(info_path, api_key)
            # تهيئة نظام RAG فوراً
            rag_system.initialize()
            models['rag'] = rag_system
            print("✅ RAG loaded and initialized successfully")
        else:
            print("❌ info.md file not found")
            # بدون إنشاء sample - نستخدم fallback بسيط
            class SimpleRAG:
                def ask_question(self, question):
                    return "❌ ملف info.md غير موجود. يرجى التأكد من وجود الملف في المسار الصحيح."
            models['rag'] = SimpleRAG()

    except Exception as e:
        print(f"❌ RAG loading failed: {e}")
        class SimpleRAG:
            def ask_question(self, question):
                return f"خطأ في تحميل نظام RAG: {str(e)}"
        models['rag'] = SimpleRAG()

    return models

# ===== Pipeline =====
def pipeline(audio_file):
    # Step 0: Convert audio file
    try:
        if isinstance(audio_file, tuple):
            audio_file = audio_file[0]
        wav_path = convert_to_wav(audio_file)
    except Exception as e:
        error_msg = f"Audio Conversion Error: {e}"
        return error_msg, "", None

    # Step 1: STT
    try:
        text = models['stt'].transcribe_audio(wav_path)
        if not text or text.strip() == "":
            text = "❌ لم يتم التعرف على أي نص في التسجيل الصوتي"
    except Exception as e:
        text = f"STT Error: {e}"

    # Step 2: RAG
    try:
        answer = models['rag'].ask_question(text)
    except Exception as e:
        answer = f"RAG Error: {e}"

    # Step 3: TTS
    try:
        audio_out = text_to_speech(answer)
    except Exception as e:
        audio_out = None
        print(f"TTS Error: {e}")

    # Clean up temporary files
    try:
        os.unlink(wav_path)
    except:
        pass

    return text, answer, audio_out

# ===== Init Models =====
print("🚀 Starting model initialization...")
models = initialize_models()

# Print final status
print("\n" + "="*50)
print("📊 Final Model Status:")
print(f"STT: {'✅ Loaded' if models['stt'].stt_model is not None else '⚠️ Using Fallback'}")
print(f"RAG: {'✅ Loaded' if hasattr(models['rag'], 'ask_question') else '❌ Failed'}")
print("="*50 + "\n")

# ===== Gradio Interface =====
iface = gr.Interface(
    fn=pipeline,
    inputs=gr.Audio(type="filepath", label="ارفع ملف صوت"),
    outputs=[
        gr.Textbox(label="📄 النص المستخرج (STT)"),
        gr.Textbox(label="🤖 إجابة الموديل (RAG)"),
        gr.Audio(label="🔊 الصوت الناتج (TTS)")
    ],
    title="🎙️ STT → RAG → TTS Demo",
    description="ارفع ملف صوت ليقوم النظام بتحويله لنص، ثم الإجابة عليه، ثم توليد الصوت"
)

print("🎉 Launching Gradio Interface...")
iface.launch(share=True)

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

🚀 Starting model initialization...
🔄 تهيئة نظام التعرف على الصوت...
🔄 جاري تحميل SaudiSTT...
✅ SaudiSTT loaded successfully
🔄 جاري تحميل RAG...
i find the file
✅ Found info.md at: /content/info.md
Time : 0.00
✅ RAG loaded and initialized successfully

📊 Final Model Status:
STT: ✅ Loaded
RAG: ✅ Loaded

🎉 Launching Gradio Interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed85afad5648bbdd57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


wasmdashai/vits-ar Model

In [ ]:
# ===== Imports =====
import gradio as gr
import torch
import soundfile as sf
import tempfile
from pydub import AudioSegment
from STT import SaudiSTT
from ArabicRAGGPT import ArabicRAGGPT_class
from transformers import VitsModel, AutoTokenizer
import os

# ===== Load TTS =====
@torch.no_grad()
def load_tts_model():
    # model_id = "facebook/mms-tts-ara"
    model_id = "wasmdashai/vits-ar"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = VitsModel.from_pretrained(model_id)
    return tokenizer, model

tts_tokenizer, tts_model = load_tts_model()

# ===== Convert audio to WAV 16kHz mono =====
def convert_to_wav(filepath):
    audio = AudioSegment.from_file(filepath)
    audio = audio.set_channels(1).set_frame_rate(16000)
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    audio.export(tmp_file.name, format="wav")
    return tmp_file.name

# ===== Text to Speech =====
def text_to_speech(text):
    inputs = tts_tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        wav = tts_model(**inputs).waveform
    wav = wav.squeeze().cpu().numpy().astype("float32")
    rate = tts_model.config.sampling_rate if hasattr(tts_model.config, "sampling_rate") else 22050
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(tmp_file.name, wav, rate)
    return tmp_file.name

# ===== Robust STT Fallback =====
class RobustSTT:
    def __init__(self):
        self.stt_model = None
        self.load_stt()

    def load_stt(self):
        try:
            print("🔄 جاري تحميل SaudiSTT...")
            self.stt_model = SaudiSTT()

            if hasattr(self.stt_model, 'transcribe_audio'):
                print("✅ SaudiSTT loaded successfully")
                return True
            else:
                print("❌ SaudiSTT missing transcribe_audio method")
                self.stt_model = None
                return False

        except Exception as e:
            print(f"❌ Failed to load SaudiSTT: {e}")
            self.stt_model = None
            return False

    def transcribe_audio(self, audio_path):
        try:
            if self.stt_model and hasattr(self.stt_model, 'transcribe_audio'):
                result = self.stt_model.transcribe_audio(audio_path)
                if result and result.strip():
                    return result

            return "هذا نص تجريبي من نظام STT الاحتياطي. يبدو أن النظام الرئيسي غير متوفر حالياً."

        except Exception as e:
            print(f"❌ STT transcription error: {e}")
            return f"نظام التعرف على الصوت يواجه مشكلة تقنية: {str(e)}"

# ===== Initialize Models =====
def initialize_models():
    models = {}

    # Initialize STT with robust fallback
    print("🔄 تهيئة نظام التعرف على الصوت...")
    models['stt'] = RobustSTT()

    # RAG - استخدام الفايل الموجود فقط
    try:
        print("🔄 جاري تحميل RAG...")
        api_key = "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"

        possible_paths = "/content/info.md"
        info_path = None

        if os.path.exists(possible_paths):
            print("i find the file")
            info_path = possible_paths

        if info_path:
            print(f"✅ Found info.md at: {info_path}")
            rag_system = ArabicRAGGPT_class(info_path, api_key)
            # تهيئة نظام RAG فوراً
            rag_system.initialize()
            models['rag'] = rag_system
            print("✅ RAG loaded and initialized successfully")
        else:
            print("❌ info.md file not found")
            # بدون إنشاء sample - نستخدم fallback بسيط
            class SimpleRAG:
                def ask_question(self, question):
                    return "❌ ملف info.md غير موجود. يرجى التأكد من وجود الملف في المسار الصحيح."
            models['rag'] = SimpleRAG()

    except Exception as e:
        print(f"❌ RAG loading failed: {e}")
        class SimpleRAG:
            def ask_question(self, question):
                return f"خطأ في تحميل نظام RAG: {str(e)}"
        models['rag'] = SimpleRAG()

    return models

# ===== Pipeline =====
def pipeline(audio_file):
    # Step 0: Convert audio file
    try:
        if isinstance(audio_file, tuple):
            audio_file = audio_file[0]
        wav_path = convert_to_wav(audio_file)
    except Exception as e:
        error_msg = f"Audio Conversion Error: {e}"
        return error_msg, "", None

    # Step 1: STT
    try:
        text = models['stt'].transcribe_audio(wav_path)
        if not text or text.strip() == "":
            text = "❌ لم يتم التعرف على أي نص في التسجيل الصوتي"
    except Exception as e:
        text = f"STT Error: {e}"

    # Step 2: RAG
    try:
        answer = models['rag'].ask_question(text)
    except Exception as e:
        answer = f"RAG Error: {e}"

    # Step 3: TTS
    try:
        audio_out = text_to_speech(answer)
    except Exception as e:
        audio_out = None
        print(f"TTS Error: {e}")

    # Clean up temporary files
    try:
        os.unlink(wav_path)
    except:
        pass

    return text, answer, audio_out

# ===== Init Models =====
print("🚀 Starting model initialization...")
models = initialize_models()

# Print final status
print("\n" + "="*50)
print("📊 Final Model Status:")
print(f"STT: {'✅ Loaded' if models['stt'].stt_model is not None else '⚠️ Using Fallback'}")
print(f"RAG: {'✅ Loaded' if hasattr(models['rag'], 'ask_question') else '❌ Failed'}")
print("="*50 + "\n")

# ===== Gradio Interface =====
iface = gr.Interface(
    fn=pipeline,
    inputs=gr.Audio(type="filepath", label="ارفع ملف صوت"),
    outputs=[
        gr.Textbox(label="📄 النص المستخرج (STT)"),
        gr.Textbox(label="🤖 إجابة الموديل (RAG)"),
        gr.Audio(label="🔊 الصوت الناتج (TTS)")
    ],
    title="🎙️ STT → RAG → TTS Demo",
    description="ارفع ملف صوت ليقوم النظام بتحويله لنص، ثم الإجابة عليه، ثم توليد الصوت"
)

print("🎉 Launching Gradio Interface...")
iface.launch(share=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

Hiiiiiii
✅ RAG loaded and initialized successfully

📊 Final Model Status:
STT: ✅ Loaded
RAG: ✅ Loaded

🎉 Launching Gradio Interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c2236fe7920622115.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
